In [ ]:
! pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

In [3]:
! sudo apt update

[sudo] password for pms: 


In [ ]:
! sudo apt-get install poppler-utils

In [ ]:
! sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

In [ ]:
! pip install unstructured-pytesseract
! pip install tesseract-ocr

In [1]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
raw_pdf_elements=partition_pdf(
    filename="/content/data/cj.pdf",                  # mandatory
    strategy="hi_res",                                 # mandatory to use ``hi_res`` strategy
    extract_images_in_pdf=True,                       # mandatory to set as ``True``
    extract_image_block_types=["Image", "Table"],          # optional
    extract_image_block_to_payload=False,                  # optional
    extract_image_block_output_dir="extracted_data",  # optional - only works when ``extract_image_block_to_payload=False``
)
     